In [103]:
import tensorflow as tf
import os
import numpy as np

# show the tensorflow package version
print(tf.__version__)

2.13.0-rc0


Load the saved models from `../02_model_training/saved_models/`. 

In [104]:
# load in the saved model created in tensorflow_model.ipynb

export_dir = '/Users/kgoldmann/Documents/Projects/AMBER/on_device_classifier/02_model_training/saved_models/1/'

# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

2023-05-25 16:16:21.994571: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-25 16:16:21.994585: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-25 16:16:21.994754: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /Users/kgoldmann/Documents/Projects/AMBER/on_device_classifier/02_model_training/saved_models/1/
2023-05-25 16:16:21.995696: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-05-25 16:16:21.995701: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /Users/kgoldmann/Documents/Projects/AMBER/on_device_classifier/02_model_training/saved_models/1/
2023-05-25 16:16:21.997969: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-05-25 16:16:22.067475: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at pat

In [105]:
# save the converted model

import pathlib
import re
tflite_model_file = re.sub("02_model_training", "03_model_compression", export_dir)
tflite_model_file = pathlib.Path(tflite_model_file + 'model.tflite')

tflite_model_file.write_bytes(tflite_model)

20154092

In [106]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)


[[-1.2502794   0.48398712  0.79153335]]


In [107]:
test_batches = tf.data.experimental.load('/Users/kgoldmann/Documents/Projects/AMBER/on_device_classifier/02_model_training/saved_models/1/test_batch', 
                                       (tf.TensorSpec(shape=(64, 300, 300, 3), dtype=tf.float32, name=None), 
                                        tf.TensorSpec(shape=(None,), dtype=tf.int32, name=None)))


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 4874675493141186020


In [108]:
#TODO: fix problem of dimensions

img1, label1 = next(iter(test_batches.take(1)))

print(input_data.shape)
print(img1.shape)

# subset the img1 to the first image
img1 = img1[0:1,:,:,:]

print(img1.shape)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 4874675493141186020


(1, 300, 300, 3)
(64, 300, 300, 3)
(1, 300, 300, 3)


In [109]:
test_batches.batch(1)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int32, name=None))>

In [111]:
from tqdm import tqdm
predictions = []

# This will report how many iterations per second, where each
# iteration is 100 predictions
test_labels, test_imgs = [], []
i=0
for img, label in tqdm(test_batches.take(100)):
    print(i) 
    img1 = img[0:1,:,:,:]
    interpreter.set_tensor(input_index, img1)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    
    test_labels.append(label.numpy()[0])
    test_imgs.append(img)
    i=i+1

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 4874675493141186020


0
1
2


100%|██████████| 4/4 [00:00<00:00, 10.88it/s]

3


In [82]:
predictions
test_labels

[0, 1, 0, 1]

In [85]:
# This will tell you how many of the predictions were correct
score = 0
for item in range(0,len(predictions)):
  prediction=np.argmax(predictions[item])
  label = test_labels[item]
  if prediction==label:
    score=score+1

print("Out of " + str(len(predictions)) + " I got " + str(score) + " correct")


Out of 4 I got 2 correct
